## Notebook for processing files from Toktarova supplementary materials

In [1]:
import pandas
import os

In [2]:
import pycountry


def search_pycountry(country_name: str) -> str | None:
    try:
        # Try to find the country
        country = pycountry.countries.search_fuzzy(country_name)[0]
        return country.alpha_2
    except LookupError:
        return None


def get_country_codes(country_names):
    country_codes = []
    for country_name in country_names:
        found_country_code = search_pycountry(country_name)
        if not (found_country_code):
            # print("Not Found:", country_name)
            found_country_code = "None"
            pass

        country_codes.append(found_country_code)
    return country_codes

In [99]:
def get_correct_columns(df_current_year):
    
    df_country_codes = pandas.DataFrame(df_current_year.loc[1]).reset_index(drop=True)
    # Remove the first row as it contains the country names
    df_country_codes.drop(index=0, inplace=True)
    df_country_codes = df_country_codes.reset_index(drop=True)
    df_country_codes.columns = ['Country Name']
    df_country_codes['Country Code'] = get_country_codes(df_country_codes['Country Name'])

    df_country_codes.loc[21,"Country Code"] = "BA"
    df_country_codes.loc[26,"Country Code"] = "MM"
    df_country_codes.loc[28,"Country Code"] = "KH"
    df_country_codes.loc[31,"Country Code"] = "CV"
    df_country_codes.loc[38,"Country Code"] = "CD"
    df_country_codes.loc[67,"Country Code"] = "GW"
    df_country_codes.loc[81,"Country Code"] = "CI"
    df_country_codes.loc[88,"Country Code"] = "KP"
    df_country_codes.loc[89,"Country Code"] = "KR"
    df_country_codes.loc[118,"Country Code"] = "VU"
    df_country_codes.loc[121,"Country Code"] = "NE" # misdetected country code
    df_country_codes.loc[151,"Country Code"] = "VC"
    df_country_codes.loc[153,"Country Code"] = "SZ"
    df_country_codes.loc[158,"Country Code"] = "TZ"
    df_country_codes.loc[163,"Country Code"] = "TR"

    return ["timestep"] + df_country_codes["Country Code"].values.tolist()

In [165]:
def extract_data_for_year(year: int) -> pandas.DataFrame:
    df_current_year = pandas.read_csv(f"./comparison/Toktarova/{year}.csv")

    df_country_codes = pandas.DataFrame(df_current_year.loc[1]).reset_index(drop=True)
    df_current_year.columns = get_correct_columns(df_current_year)

    # Drop rows of different 
    df_current_year.drop(index=[0,1,2,3], inplace=True)
    df_current_year.reset_index(drop=True, inplace=True)
    
    df_current_year["timestep"] = [int(str_timestep.split(" ")[0].split("_")[-1]) - 1 for str_timestep in df_current_year["timestep"]]

    df_current_year.insert(0, "year", year)

    return df_current_year
    

In [166]:
years_available = sorted([file.split(".")[0] for file in os.listdir("./comparison/Toktarova/") if file.endswith(".csv")])
print(years_available)


['2020', '2030', '2040', '2050', '2060', '2070', '2080', '2090', '2100']


#### 2020

In [167]:
extract_2020 = extract_data_for_year(2020)
print(extract_2020.shape)
extract_2020.head()

/tmp/ipykernel_11108/3781316591.py:2: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178) have mixed types. Specify dtype option on import or set low_memory=False.
  df_current_year = pandas.read_csv(f"./comparison/Toktarova/{year}.csv")


(8760, 180)


,year,timestep,AF,AL,DZ,AO,AG,AR,AM,AU,...,GB,US,UY,UZ,VE,VN,YE,RS,ZM,ZW
0,2020,0,708,6741472,932,6792476,6078,437955,3431,831925,...,1526,408993,14398,35971,2617,568617,2,655452279,1758,19363
1,2020,1,686,6610016,934,3454279,5619,712832,3363,349699,...,1428,756356,14162,4501,2571,4094,2,479960676,1686,342535
2,2020,2,678,3324312,957,744994,5448,251906,3377,112651,...,1392,710297,14085,19528,2555,869209,2,418665621,1661,998639
3,2020,3,685,2494948,1000,173056,5596,825743,3477,392763,...,1425,299029,14184,65307,2575,621853,2,484583111,1690,379313
4,2020,4,705,7610508,1059,289667,6030,750766,3651,253753,...,1519,78881,14441,5673,2626,492487,2,663881716,1765,746366


#### 2030

In [168]:
extract_2030 = extract_data_for_year(2030)
print(extract_2030.shape)
extract_2030.head()

/tmp/ipykernel_11108/3781316591.py:2: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178) have mixed types. Specify dtype option on import or set low_memory=False.
  df_current_year = pandas.read_csv(f"./comparison/Toktarova/{year}.csv")


(8760, 180)


,year,timestep,AF,AL,DZ,AO,AG,AR,AM,AU,...,GB,US,UY,UZ,VE,VN,YE,RS,ZM,ZW
0,2030,0,2415,194229,1511,593502,13363,2738,11815,2737,...,2373,97384,20154,72437,10998,89652,7,380017084,2846,313384
1,2030,1,2345,405265,1513,86533,12365,79651,11577,44066,...,2226,111999,19819,81025,10801,70439,6,896183023,2730,460782
2,2030,2,2321,939736,1550,201903,12023,41945,11619,34179,...,2174,167808,19706,45573,10735,11939,6,725161147,2690,291372
3,2030,3,2350,72622,1619,484568,12410,50479,11955,60964,...,2229,75518,19840,3543,10814,10251,6,902694398,2734,461736
4,2030,4,2424,185686,1714,663239,13447,68921,12542,16718,...,2379,220178,20193,61266,11022,77066,7,390869413,2853,786457


#### 2040

In [169]:
extract_2040 = extract_data_for_year(2040)
print(extract_2040.shape)
extract_2040.head()

/tmp/ipykernel_11108/3781316591.py:2: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178) have mixed types. Specify dtype option on import or set low_memory=False.
  df_current_year = pandas.read_csv(f"./comparison/Toktarova/{year}.csv")


(8760, 180)


,year,timestep,AF,AL,DZ,AO,AG,AR,AM,AU,...,GB,US,UY,UZ,VE,VN,YE,RS,ZM,ZW
0,2040,0,8002,408188,2101,753976,23299,51319,26816,61247,...,3704,100211,25686,54325,42869,2591,22,64750154,7224,875873
1,2040,1,7751,755364,2103,515028,21319,75687,26221,24949,...,3459,104342,25229,56966,42046,67286,21,10946918,6915,35781
2,2040,2,7659,903984,2155,755534,20596,92299,26302,47894,...,3369,938177,25068,96501,41757,65221,20,55812013,6805,427977
3,2040,3,7744,973789,2256,82847,21274,42521,27095,37443,...,3454,38103,25238,30141,42062,61595,21,10570004,6917,74938
4,2040,4,7987,818727,2396,358571,23200,58916,28494,18229,...,3693,610702,25701,904,42897,35332,22,63309902,7228,258382


#### 2050

In [170]:
extract_2050 = extract_data_for_year(2050)
print(extract_2050.shape)
extract_2050.head()

/tmp/ipykernel_11108/3781316591.py:2: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178) have mixed types. Specify dtype option on import or set low_memory=False.
  df_current_year = pandas.read_csv(f"./comparison/Toktarova/{year}.csv")


(8760, 180)


,year,timestep,AF,AL,DZ,AO,AG,AR,AM,AU,...,GB,US,UY,UZ,VE,VN,YE,RS,ZM,ZW
0,2050,0,22798,51646,2736,553439,33957,99055,54549,63022,...,5580,172292,32733,4161,107103,213,69,96795346,18551,26172
1,2050,1,22076,28089,2738,204722,30943,7142,53369,27368,...,5201,632897,32156,56867,105070,8235,65,48328783,17792,4184
2,2050,2,21819,97914,2803,949798,29867,67634,53506,37519,...,5065,732478,31947,90164,104334,8281,63,85725045,17517,74013
3,2050,3,21300,44916,2743,891134,28108,70827,51744,75868,...,4946,216978,31495,64109,102732,6859,62,3122322,17018,79869
4,2050,4,22060,94799,2920,8702,31226,29095,54503,21124,...,5329,403888,32098,10953,104858,7893,66,53902379,17795,18786


#### 2060

In [171]:
extract_2060 = extract_data_for_year(2060)
print(extract_2060.shape)
extract_2060.head()

(1463, 180)


,year,timestep,AF,AL,DZ,AO,AG,AR,AM,AU,...,GB,US,UY,UZ,VE,VN,YE,RS,ZM,ZW
0,2060,0,45457,26869,3121,122028,43653,42599,92095,3912,...,8122,383967,38584,1849,167457,5077,142,5175986,31593,28447
1,2060,1,44017,22515,3123,5367,39777,70934,90102,26702,...,7571,389813,37904,6832,164280,1298,133,3827913,30300,95443
2,2060,2,43506,19289,3197,989616,38395,27538,90333,73272,...,7373,57596,37658,10521,163129,3876,130,707187,29833,17015
3,2060,3,42470,31785,3129,490892,36134,997,87359,63115,...,7199,611657,37125,994,160624,4095,126,3512871,28983,45982
4,2060,4,43986,65336,3331,340874,40141,79164,92016,67089,...,7757,370639,37835,1605,163948,6105,135,533218,30305,6709


#### 2070

In [172]:
extract_2070 = extract_data_for_year(2070)
print(extract_2070.shape)
extract_2070.head()

(1687, 180)


,year,timestep,AF,AL,DZ,AO,AG,AR,AM,AU,...,GB,US,UY,UZ,VE,VN,YE,RS,ZM,ZW
0,2070,0,66121,54644,3342,940968,54030,18085,139963,5306,...,11074,17431,44495,57928,218063,3412,217,1758329,42655,88743
1,2070,1,64018,42613,3349,50909,49525,84897,137238,6681,...,10321,9535,43761,8421,214218,6856,203,9728236,41019,6187
2,2070,2,63222,72043,3432,399199,47842,2385,137786,2869,...,10043,11485,43521,56051,212965,216,199,3613255,40463,14008
3,2070,3,63883,57177,3590,443096,49300,46359,141776,3663,...,10294,43308,43830,89883,214597,2137,204,3205898,41111,24649
4,2070,4,65843,23128,3807,22575,53560,62718,148694,1802,...,11017,64585,44629,96558,218800,7346,217,8100167,42832,34272


#### 2080

In [173]:
extract_2080 = extract_data_for_year(2080)
print(extract_2080.shape)
extract_2080.head()

(1755, 180)


,year,timestep,AF,AL,DZ,AO,AG,AR,AM,AU,...,GB,US,UY,UZ,VE,VN,YE,RS,ZM,ZW
0,2080,0,76347,6258,3295,915576,58008,60432,186511,48,...,13755,75119,48755,87259,255172,2093,267,8226025,48125,4567
1,2080,1,73919,24578,3301,939567,53172,60337,182879,9386,...,12820,3165,47951,88268,250673,2906,251,5405684,46278,35323
2,2080,2,73000,47959,3384,115391,51364,79256,183609,6781,...,12475,2388,47688,59498,249206,2846,245,8536401,45651,15349
3,2080,3,73763,53541,3539,936073,52930,62211,188926,7326,...,12787,19804,48027,55134,251116,2346,251,9694359,46382,35738
4,2080,4,76026,26759,3753,66923,57504,47584,198145,1942,...,13685,53453,48903,1259,256035,884,268,604682,48324,12532


#### 2090

In [174]:
extract_2090 = extract_data_for_year(2090)
print(extract_2090.shape)
extract_2090.head()

/tmp/ipykernel_11108/3781316591.py:2: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178) have mixed types. Specify dtype option on import or set low_memory=False.
  df_current_year = pandas.read_csv(f"./comparison/Toktarova/{year}.csv")


(8760, 180)


,year,timestep,AF,AL,DZ,AO,AG,AR,AM,AU,...,GB,US,UY,UZ,VE,VN,YE,RS,ZM,ZW
0,2090,0,77488,64906,3072,573406,58019,75751,230215,1158,...,15211,72542,51478,7496,283149,733,299,4878818,50352,84089
1,2090,1,75031,99608,3069,780846,53100,65349,225007,9885,...,14148,88712,50527,89419,277539,7454,280,4414005,48271,30666
2,2090,2,74140,27332,3138,179142,51317,23403,225145,3682,...,13763,64876,50147,78272,275282,924,273,2328321,47463,89629
3,2090,3,74991,94034,3275,451881,53027,12652,230908,286,...,14133,30158,50403,18339,276764,3276,279,2161022,48076,57214
4,2090,4,77397,32706,3466,896257,57850,4978,241457,112,...,15175,81686,51225,9806,281578,944,296,9590118,49955,15682


#### 2100

In [175]:
extract_2100 = extract_data_for_year(2100)
print(extract_2100.shape)
extract_2100.head()

/tmp/ipykernel_11108/3781316591.py:2: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178) have mixed types. Specify dtype option on import or set low_memory=False.
  df_current_year = pandas.read_csv(f"./comparison/Toktarova/{year}.csv")


(8760, 180)


,year,timestep,AF,AL,DZ,AO,AG,AR,AM,AU,...,GB,US,UY,UZ,VE,VN,YE,RS,ZM,ZW
0,2100,0,76488,16402,2839,484842,58561,8596,266211,4312,...,16467,15213,52643,65645,302817,7981,319,7221653,50799,97509
1,2100,1,74063,22981,2836,904128,53596,8635,260190,1202,...,15316,59757,51671,96141,296818,1311,299,3888476,48699,9568
2,2100,2,73183,2041,2900,11366,51796,2746,260348,9805,...,14899,56542,51283,24335,294404,5471,291,6932471,47885,3766
3,2100,3,74023,69124,3026,972748,53521,87337,267012,9852,...,15299,72576,51544,42688,295988,8515,298,807646,48503,49302
4,2100,4,76398,2111,3203,893958,58390,24704,279211,22,...,16428,28004,52384,95157,301136,9908,317,224373,50398,75957


#### Combine years

In [176]:
extract_all_years = pandas.concat([
    extract_2020,
    extract_2030,
    extract_2040,
    extract_2050,
    extract_2060,
    extract_2070,
    extract_2080,
    extract_2090,
    extract_2100
    ], ignore_index=True)


In [178]:
extract_all_years.shape

(57465, 180)

In [179]:
extract_all_years.dtypes

year         int64
timestep     int64
AF          object
AL          object
DZ          object
             ...  
VN          object
YE          object
RS          object
ZM          object
ZW          object
Length: 180, dtype: object

In [177]:
extract_all_years.head()

,year,timestep,AF,AL,DZ,AO,AG,AR,AM,AU,...,GB,US,UY,UZ,VE,VN,YE,RS,ZM,ZW
0,2020,0,708,6741472,932,6792476,6078,437955,3431,831925,...,1526,408993,14398,35971,2617,568617,2,655452279,1758,19363
1,2020,1,686,6610016,934,3454279,5619,712832,3363,349699,...,1428,756356,14162,4501,2571,4094,2,479960676,1686,342535
2,2020,2,678,3324312,957,744994,5448,251906,3377,112651,...,1392,710297,14085,19528,2555,869209,2,418665621,1661,998639
3,2020,3,685,2494948,1000,173056,5596,825743,3477,392763,...,1425,299029,14184,65307,2575,621853,2,484583111,1690,379313
4,2020,4,705,7610508,1059,289667,6030,750766,3651,253753,...,1519,78881,14441,5673,2626,492487,2,663881716,1765,746366


In [ ]:
for col in tqdm(extract_all_years.columns):
    extract_all_years[col] = extract_all_years[col].apply(int)



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 180/180 [00:04<00:00, 40.77it/s]


In [186]:
extract_all_years.dtypes

year        int64
timestep    int64
AF          int64
AL          int64
DZ          int64
            ...  
VN          int64
YE          int64
RS          int64
ZM          int64
ZW          int64
Length: 180, dtype: object

In [187]:
extract_all_years.head()

,year,timestep,AF,AL,DZ,AO,AG,AR,AM,AU,...,GB,US,UY,UZ,VE,VN,YE,RS,ZM,ZW
0,2020,0,708,6741472,932,6792476,6078,437955,3431,831925,...,1526,408993,14398,35971,2617,568617,2,655452279,1758,19363
1,2020,1,686,6610016,934,3454279,5619,712832,3363,349699,...,1428,756356,14162,4501,2571,4094,2,479960676,1686,342535
2,2020,2,678,3324312,957,744994,5448,251906,3377,112651,...,1392,710297,14085,19528,2555,869209,2,418665621,1661,998639
3,2020,3,685,2494948,1000,173056,5596,825743,3477,392763,...,1425,299029,14184,65307,2575,621853,2,484583111,1690,379313
4,2020,4,705,7610508,1059,289667,6030,750766,3651,253753,...,1519,78881,14441,5673,2626,492487,2,663881716,1765,746366


In [190]:
extract_all_years.max().max()

999791752

In [ ]:
extract_all_years.to_parquet("./comparison/Toktarova/all_years.parquet", )

In [161]:
df_current_year = pandas.read_csv(f"./comparison/Toktarova/{2020}.csv")

/tmp/ipykernel_11108/1585790131.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178) have mixed types. Specify dtype option on import or set low_memory=False.
  df_current_year = pandas.read_csv(f"./comparison/Toktarova/{2020}.csv")


In [162]:
df_current_year.head()

,2020,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 169,Unnamed: 170,Unnamed: 171,Unnamed: 172,Unnamed: 173,Unnamed: 174,Unnamed: 175,Unnamed: 176,Unnamed: 177,Unnamed: 178
0,Countrynumber,1,2,3,5,7,9,10,11,12,...,193,194,196,197,198,199,203,204,205,206
1,Countryname,Afghanistan,Albania,Algeria,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,United States,Uruguay,Uzbekistan,Venezuela,Vietnam,Yemen,Serbia,Zambia,Zimbabwe
2,annual electricity consumption in TWh,9,19186995,10,11532643,99,80658614,33,2079296,0,...,21,41331197,120,9812206,24,66870704,0,3972566,16,46855995
3,synthetic peak in MW,1532,950566,1639,319734,17861,83257,5065,420414,58,...,3952,967143,17490,79138,3500,625734,7,912606709,3461,676783
4,Hour_1 in MW,708,6741472,932,6792476,6078,437955,3431,831925,39,...,1526,408993,14398,35971,2617,568617,2,655452279,1758,19363


In [163]:
df_current_year = pandas.read_csv(f"./comparison/Toktarova/{2020}.csv")

df_country_codes = pandas.DataFrame(df_current_year.loc[1]).reset_index(drop=True)
df_current_year.columns = get_correct_columns(df_current_year)

# Drop rows of different 
df_current_year.drop(index=[0,1,2,3], inplace=True)
df_current_year.reset_index(drop=True, inplace=True)

df_current_year["timestep"] = [int(str_timestep.split(" ")[0].split("_")[-1]) - 1 for str_timestep in df_current_year["timestep"]]

df_current_year.insert(0, "year", 2020)

/tmp/ipykernel_11108/1809090018.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178) have mixed types. Specify dtype option on import or set low_memory=False.
  df_current_year = pandas.read_csv(f"./comparison/Toktarova/{2020}.csv")


In [164]:
df_current_year

,year,timestep,AF,AL,DZ,AO,AG,AR,AM,AU,...,GB,US,UY,UZ,VE,VN,YE,RS,ZM,ZW
0,2020,0,708,6741472,932,6792476,6078,437955,3431,831925,...,1526,408993,14398,35971,2617,568617,2,655452279,1758,19363
1,2020,1,686,6610016,934,3454279,5619,712832,3363,349699,...,1428,756356,14162,4501,2571,4094,2,479960676,1686,342535
2,2020,2,678,3324312,957,744994,5448,251906,3377,112651,...,1392,710297,14085,19528,2555,869209,2,418665621,1661,998639
3,2020,3,685,2494948,1000,173056,5596,825743,3477,392763,...,1425,299029,14184,65307,2575,621853,2,484583111,1690,379313
4,2020,4,705,7610508,1059,289667,6030,750766,3651,253753,...,1519,78881,14441,5673,2626,492487,2,663881716,1765,746366
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2020,8755,1118,282189,1396,191305,10854,95502,4661,404819,...,2626,364953,17428,12286,3430,92025,5,748686515,3461,676783
8756,2020,8756,1092,775096,1354,38141,10765,54406,4924,604077,...,2624,230601,16962,27482,3459,333783,5,857545316,3249,351417
8757,2020,8757,967,9359401,1213,976022,9537,62787,4676,838168,...,2336,451957,16236,121,3238,917345,4,979179747,2739,79355
8758,2020,8758,818,4106153,1054,297922,7845,815876,4060,505112,...,1931,583305,15482,96496,2919,428857,3,715830914,2198,200373
